# M5 Walmart Sales Forecasting - Model Evaluation

This notebook provides comprehensive evaluation and analysis of the trained forecasting models.

## Evaluation Components

1. **Model Performance Analysis**: Detailed metrics and comparisons
2. **Residual Analysis**: Understanding prediction errors
3. **Feature Importance**: What drives the predictions
4. **Seasonal Decomposition**: Understanding model behavior
5. **Business Impact**: Translation to business metrics
6. **Model Diagnostics**: Statistical validation

In [ ]:
# Import necessary libraries
import sys
import os
import warnings

# Add src to path for imports
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
from datetime import datetime, timedelta

# Model imports
from src.models.sarima_model import SarimaModel
from src.models.lstm_model import LSTMModel
from src.models.prophet_model import ProphetModel

# Utility imports
from src.visualization.plots import M5Visualizer
from src.utils.config import get_config
from src.utils.logger import setup_logger
from src.utils.metrics import calculate_metrics

# Setup
warnings.filterwarnings('ignore')
logger = setup_logger('model_evaluation')
config = get_config()
visualizer = M5Visualizer()

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Load Training Results and Data

In [ ]:
# Load training results and metadata
models_dir = config.get('models.model_path', 'models/')
processed_data_path = config.get('data.processed_data_path', 'data/processed/')

print("Loading training results and metadata...")

# Load model comparison results
try:
    results_df = pd.read_csv(f"{models_dir}/model_comparison_results.csv")
    print(f"✅ Loaded model comparison results: {results_df.shape}")
    print(results_df)
except FileNotFoundError:
    print("❌ Model comparison results not found. Please run 03_model_training.ipynb first.")
    raise

# Load training metadata
try:
    with open(f"{models_dir}/training_metadata.json", 'r') as f:
        training_metadata = json.load(f)
    print(f"✅ Loaded training metadata")
    
    highest_selling_product = training_metadata['product_id']
    best_model = training_metadata['best_model']
    print(f"   Product: {highest_selling_product}")
    print(f"   Best model: {best_model}")
    
except FileNotFoundError:
    print("❌ Training metadata not found. Using defaults.")
    highest_selling_product = "HOBBIES_1_001_CA_1_validation"
    best_model = "SARIMA"

# Load processed data
try:
    sales_processed = pd.read_parquet(f"{processed_data_path}/sales_processed.parquet")
    product_ts = pd.read_csv(f"{processed_data_path}/highest_selling_product_ts.csv", 
                            index_col=0, parse_dates=True)
    print(f"✅ Loaded processed data: {sales_processed.shape}, {product_ts.shape}")
except FileNotFoundError:
    print("❌ Processed data not found. Please run feature engineering first.")
    raise

## 2. Recreate Train/Test Splits and Predictions

In [ ]:
# Recreate the train/test splits used in training
train_end_date = training_metadata['data_splits']['train_end_date']
val_end_date = training_metadata['data_splits']['val_end_date']

print(f"Recreating data splits:")
print(f"  Train end: {train_end_date}")
print(f"  Validation end: {val_end_date}")

# Clean time series data
product_ts_clean = product_ts.dropna(subset=['sales'])

# Create splits
train_data = product_ts_clean[product_ts_clean.index <= train_end_date]
val_data = product_ts_clean[(product_ts_clean.index > train_end_date) & 
                          (product_ts_clean.index <= val_end_date)]
test_data = product_ts_clean[product_ts_clean.index > val_end_date]

print(f"\nData split sizes:")
print(f"  Training: {len(train_data)} days")
print(f"  Validation: {len(val_data)} days")
print(f"  Test: {len(test_data)} days")

# Extract target values
y_train = train_data['sales'].values
y_val = val_data['sales'].values
y_test = test_data['sales'].values

## 3. Load Trained Models and Generate Predictions

In [ ]:
# Load and evaluate SARIMA model
print("Loading and evaluating SARIMA model...")
try:
    sarima_model = SarimaModel()
    sarima_model.load_model(f"{models_dir}/sarima_model.pkl")
    
    # Generate predictions
    sarima_val_pred = sarima_model.predict(len(y_val))
    
    # For test predictions, retrain on train+val
    y_train_val = np.concatenate([y_train, y_val])
    sarima_model_full = SarimaModel()
    sarima_model_full.fit(y_train_val, seasonal_periods=7)
    sarima_test_pred = sarima_model_full.predict(len(y_test))
    
    print("✅ SARIMA predictions generated")
    sarima_available = True
    
except Exception as e:
    print(f"❌ SARIMA model loading failed: {e}")
    sarima_val_pred = np.zeros_like(y_val)
    sarima_test_pred = np.zeros_like(y_test)
    sarima_available = False

In [ ]:
# Load and evaluate LSTM model
print("Loading and evaluating LSTM model...")
try:
    lstm_model = LSTMModel(sequence_length=28, n_features=1)
    lstm_model.load_model(f"{models_dir}/lstm_model.h5")
    
    # Prepare features for LSTM
    feature_columns = ['sales']
    X_train = train_data[feature_columns].fillna(0).values
    X_val = val_data[feature_columns].fillna(0).values
    X_test = test_data[feature_columns].fillna(0).values
    
    # Generate predictions
    lstm_val_pred = lstm_model.predict(X_val)
    lstm_test_pred = lstm_model.predict(X_test)
    
    print("✅ LSTM predictions generated")
    lstm_available = True
    
except Exception as e:
    print(f"❌ LSTM model loading failed: {e}")
    lstm_val_pred = np.zeros_like(y_val)
    lstm_test_pred = np.zeros_like(y_test)
    lstm_available = False

In [ ]:
# Load and evaluate Prophet model
print("Loading and evaluating Prophet model...")
try:
    prophet_model = ProphetModel()
    prophet_model.load_model(f"{models_dir}/prophet_model.pkl")
    
    # Generate predictions
    prophet_val_pred = prophet_model.predict(len(y_val))['yhat'].values
    
    # For test predictions, retrain on train+val
    prophet_train_val = pd.DataFrame({
        'ds': pd.concat([train_data, val_data]).index,
        'y': np.concatenate([y_train, y_val])
    })
    prophet_model_full = ProphetModel()
    prophet_model_full.fit(prophet_train_val)
    prophet_test_pred = prophet_model_full.predict(len(y_test))['yhat'].values
    
    print("✅ Prophet predictions generated")
    prophet_available = True
    
except Exception as e:
    print(f"❌ Prophet model loading failed: {e}")
    prophet_val_pred = np.zeros_like(y_val)
    prophet_test_pred = np.zeros_like(y_test)
    prophet_available = False

## 4. Detailed Performance Analysis

In [ ]:
# Calculate comprehensive metrics for all models
print("\n📊 COMPREHENSIVE PERFORMANCE ANALYSIS")
print("=" * 45)

def extended_metrics(y_true, y_pred, model_name):
    """Calculate extended metrics for model evaluation"""
    metrics = calculate_metrics(y_true, y_pred)
    
    # Additional metrics
    metrics['mean_error'] = np.mean(y_pred - y_true)
    metrics['std_error'] = np.std(y_pred - y_true)
    metrics['max_error'] = np.max(np.abs(y_pred - y_true))
    metrics['r2_score'] = 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))
    
    # Direction accuracy (for forecasting)
    if len(y_true) > 1:
        true_direction = np.diff(y_true) > 0
        pred_direction = np.diff(y_pred) > 0
        metrics['direction_accuracy'] = np.mean(true_direction == pred_direction) * 100
    else:
        metrics['direction_accuracy'] = np.nan
    
    return metrics

# Calculate metrics for all models
models_predictions = {
    'SARIMA': {'val': sarima_val_pred, 'test': sarima_test_pred, 'available': sarima_available},
    'LSTM': {'val': lstm_val_pred, 'test': lstm_test_pred, 'available': lstm_available},
    'Prophet': {'val': prophet_val_pred, 'test': prophet_test_pred, 'available': prophet_available}
}

detailed_results = []

for model_name, preds in models_predictions.items():
    if preds['available']:
        val_metrics = extended_metrics(y_val, preds['val'], model_name)
        test_metrics = extended_metrics(y_test, preds['test'], model_name)
        
        detailed_results.append({
            'Model': model_name,
            'Dataset': 'Validation',
            **val_metrics
        })
        
        detailed_results.append({
            'Model': model_name,
            'Dataset': 'Test',
            **test_metrics
        })

detailed_df = pd.DataFrame(detailed_results)
print(detailed_df.round(3))

In [ ]:
# Visualize detailed performance metrics
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

metrics_to_plot = ['rmse', 'mae', 'mape', 'r2_score', 'direction_accuracy', 'max_error']
metric_titles = ['RMSE', 'MAE', 'MAPE (%)', 'R² Score', 'Direction Accuracy (%)', 'Max Error']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, metric_titles)):
    row = idx // 3
    col = idx % 3
    
    # Filter data for this metric
    test_data_metric = detailed_df[detailed_df['Dataset'] == 'Test']
    
    if metric in test_data_metric.columns:
        axes[row, col].bar(test_data_metric['Model'], test_data_metric[metric])
        axes[row, col].set_title(f'Test {title}')
        axes[row, col].set_ylabel(title)
        
        # Rotate x-axis labels if needed
        if len(test_data_metric) > 3:
            axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Residual Analysis

In [ ]:
# Residual analysis for each model
print("\n🔍 RESIDUAL ANALYSIS")
print("=" * 25)

def analyze_residuals(y_true, y_pred, model_name):
    """Analyze prediction residuals"""
    residuals = y_true - y_pred
    
    print(f"\n{model_name} Residuals:")
    print(f"  Mean: {np.mean(residuals):.3f}")
    print(f"  Std: {np.std(residuals):.3f}")
    print(f"  Skewness: {stats.skew(residuals):.3f}")
    print(f"  Kurtosis: {stats.kurtosis(residuals):.3f}")
    
    # Normality test
    _, p_value = stats.normaltest(residuals)
    print(f"  Normality test p-value: {p_value:.3f}")
    print(f"  Residuals normal: {'Yes' if p_value > 0.05 else 'No'}")
    
    return residuals

# Analyze residuals for available models
residuals_dict = {}

for model_name, preds in models_predictions.items():
    if preds['available']:
        residuals_dict[model_name] = analyze_residuals(y_test, preds['test'], model_name)

In [ ]:
# Visualize residuals
available_models = [name for name, preds in models_predictions.items() if preds['available']]
n_models = len(available_models)

if n_models > 0:
    fig, axes = plt.subplots(n_models, 3, figsize=(15, 5 * n_models))
    
    if n_models == 1:
        axes = axes.reshape(1, -1)
    
    for idx, model_name in enumerate(available_models):
        residuals = residuals_dict[model_name]
        
        # Residuals over time
        axes[idx, 0].plot(test_data.index, residuals)
        axes[idx, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
        axes[idx, 0].set_title(f'{model_name} - Residuals Over Time')
        axes[idx, 0].set_xlabel('Date')
        axes[idx, 0].set_ylabel('Residuals')
        
        # Residuals histogram
        axes[idx, 1].hist(residuals, bins=20, alpha=0.7, edgecolor='black')
        axes[idx, 1].axvline(x=np.mean(residuals), color='red', linestyle='--', label='Mean')
        axes[idx, 1].set_title(f'{model_name} - Residuals Distribution')
        axes[idx, 1].set_xlabel('Residuals')
        axes[idx, 1].set_ylabel('Frequency')
        axes[idx, 1].legend()
        
        # Q-Q plot
        stats.probplot(residuals, dist="norm", plot=axes[idx, 2])
        axes[idx, 2].set_title(f'{model_name} - Q-Q Plot')
        axes[idx, 2].grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("No models available for residual analysis")

## 6. Error Analysis by Time and Magnitude

In [ ]:
# Error analysis by different dimensions
print("\n📈 ERROR ANALYSIS")
print("=" * 20)

def error_analysis_by_dimension(y_true, y_pred, dates, model_name):
    """Analyze errors by different dimensions"""
    errors = np.abs(y_true - y_pred)
    
    # Error by day of week
    df_temp = pd.DataFrame({
        'date': dates,
        'error': errors,
        'actual': y_true,
        'predicted': y_pred
    })
    
    df_temp['day_of_week'] = df_temp['date'].dt.day_name()
    df_temp['week'] = df_temp['date'].dt.isocalendar().week
    df_temp['month'] = df_temp['date'].dt.month
    
    print(f"\n{model_name} Error Analysis:")
    print("Error by Day of Week:")
    dow_errors = df_temp.groupby('day_of_week')['error'].mean().sort_values(ascending=False)
    print(dow_errors.round(3))
    
    print("\nError by Sales Magnitude:")
    df_temp['sales_quartile'] = pd.qcut(df_temp['actual'], q=4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
    quartile_errors = df_temp.groupby('sales_quartile')['error'].mean()
    print(quartile_errors.round(3))
    
    return df_temp

# Perform error analysis for available models
error_analysis_results = {}

for model_name, preds in models_predictions.items():
    if preds['available']:
        error_analysis_results[model_name] = error_analysis_by_dimension(
            y_test, preds['test'], test_data.index, model_name
        )

In [ ]:
# Visualize error patterns
if error_analysis_results:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Error by day of week
    for model_name, df_temp in error_analysis_results.items():
        dow_errors = df_temp.groupby('day_of_week')['error'].mean()
        axes[0, 0].plot(dow_errors.index, dow_errors.values, marker='o', label=model_name)
    
    axes[0, 0].set_title('Average Error by Day of Week')
    axes[0, 0].set_xlabel('Day of Week')
    axes[0, 0].set_ylabel('Mean Absolute Error')
    axes[0, 0].legend()
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Error by sales magnitude
    for model_name, df_temp in error_analysis_results.items():
        quartile_errors = df_temp.groupby('sales_quartile')['error'].mean()
        axes[0, 1].plot(quartile_errors.index, quartile_errors.values, marker='o', label=model_name)
    
    axes[0, 1].set_title('Average Error by Sales Quartile')
    axes[0, 1].set_xlabel('Sales Quartile')
    axes[0, 1].set_ylabel('Mean Absolute Error')
    axes[0, 1].legend()
    
    # Error over time
    for model_name, df_temp in error_analysis_results.items():
        axes[1, 0].plot(df_temp['date'], df_temp['error'], alpha=0.7, label=model_name)
    
    axes[1, 0].set_title('Error Over Time')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Absolute Error')
    axes[1, 0].legend()
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Actual vs Predicted scatter
    for model_name, df_temp in error_analysis_results.items():
        axes[1, 1].scatter(df_temp['actual'], df_temp['predicted'], alpha=0.6, label=model_name)
    
    # Perfect prediction line
    min_val = min(y_test.min(), min([preds['test'].min() for preds in models_predictions.values() if preds['available']]))
    max_val = max(y_test.max(), max([preds['test'].max() for preds in models_predictions.values() if preds['available']]))
    axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.7, label='Perfect Prediction')
    
    axes[1, 1].set_title('Actual vs Predicted')
    axes[1, 1].set_xlabel('Actual Sales')
    axes[1, 1].set_ylabel('Predicted Sales')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

## 7. Business Impact Analysis

In [ ]:
# Business impact analysis
print("\n💼 BUSINESS IMPACT ANALYSIS")
print("=" * 30)

def business_impact_analysis(y_true, y_pred, model_name):
    """Analyze business impact of predictions"""
    
    # Assume average product price and profit margins
    avg_price = 10.0  # Average price per unit
    profit_margin = 0.2  # 20% profit margin
    
    # Calculate revenue impact
    actual_revenue = np.sum(y_true) * avg_price
    predicted_revenue = np.sum(y_pred) * avg_price
    revenue_error = predicted_revenue - actual_revenue
    revenue_error_pct = (revenue_error / actual_revenue) * 100
    
    # Calculate inventory impact (overstocking/understocking)
    inventory_error = np.sum(np.abs(y_pred - y_true))
    inventory_cost = inventory_error * avg_price * 0.1  # 10% carrying cost
    
    # Lost sales (understocking)
    understocking = np.sum(np.maximum(0, y_true - y_pred))
    lost_sales_cost = understocking * avg_price * profit_margin
    
    # Overstocking cost
    overstocking = np.sum(np.maximum(0, y_pred - y_true))
    overstocking_cost = overstocking * avg_price * 0.05  # 5% markdown cost
    
    print(f"\n{model_name} Business Impact:")
    print(f"  Actual revenue: ${actual_revenue:,.2f}")
    print(f"  Predicted revenue: ${predicted_revenue:,.2f}")
    print(f"  Revenue error: ${revenue_error:,.2f} ({revenue_error_pct:.2f}%)")
    print(f"  Inventory carrying cost: ${inventory_cost:,.2f}")
    print(f"  Lost sales cost: ${lost_sales_cost:,.2f}")
    print(f"  Overstocking cost: ${overstocking_cost:,.2f}")
    print(f"  Total operational cost: ${inventory_cost + lost_sales_cost + overstocking_cost:,.2f}")
    
    return {
        'actual_revenue': actual_revenue,
        'predicted_revenue': predicted_revenue,
        'revenue_error': revenue_error,
        'revenue_error_pct': revenue_error_pct,
        'inventory_cost': inventory_cost,
        'lost_sales_cost': lost_sales_cost,
        'overstocking_cost': overstocking_cost,
        'total_cost': inventory_cost + lost_sales_cost + overstocking_cost
    }

# Calculate business impact for available models
business_impacts = {}

for model_name, preds in models_predictions.items():
    if preds['available']:
        business_impacts[model_name] = business_impact_analysis(y_test, preds['test'], model_name)

In [ ]:
# Visualize business impact
if business_impacts:
    business_df = pd.DataFrame(business_impacts).T
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Revenue error
    axes[0].bar(business_df.index, business_df['revenue_error_pct'])
    axes[0].set_title('Revenue Prediction Error (%)')
    axes[0].set_ylabel('Error Percentage')
    axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    # Total operational cost
    axes[1].bar(business_df.index, business_df['total_cost'])
    axes[1].set_title('Total Operational Cost ($)')
    axes[1].set_ylabel('Cost ($)')
    
    # Cost breakdown
    cost_components = ['inventory_cost', 'lost_sales_cost', 'overstocking_cost']
    bottom = np.zeros(len(business_df))
    
    for component in cost_components:
        axes[2].bar(business_df.index, business_df[component], bottom=bottom, 
                   label=component.replace('_', ' ').title())
        bottom += business_df[component]
    
    axes[2].set_title('Cost Breakdown')
    axes[2].set_ylabel('Cost ($)')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print business impact summary
    print("\n💰 BUSINESS IMPACT SUMMARY:")
    best_business_model = business_df['total_cost'].idxmin()
    print(f"Most cost-effective model: {best_business_model}")
    print(f"Lowest operational cost: ${business_df.loc[best_business_model, 'total_cost']:,.2f}")

## 8. Model Diagnostics and Validation

In [ ]:
# Statistical validation tests
print("\n🔬 MODEL DIAGNOSTICS")
print("=" * 25)

def statistical_validation(y_true, y_pred, model_name):
    """Perform statistical validation tests"""
    residuals = y_true - y_pred
    
    print(f"\n{model_name} Statistical Tests:")
    
    # Ljung-Box test for residual autocorrelation
    try:
        from statsmodels.stats.diagnostic import acorr_ljungbox
        lb_stat, lb_pvalue = acorr_ljungbox(residuals, lags=10, return_df=False)
        print(f"  Ljung-Box test p-value: {lb_pvalue[-1]:.3f}")
        print(f"  Residuals independent: {'Yes' if lb_pvalue[-1] > 0.05 else 'No'}")
    except ImportError:
        print("  Ljung-Box test: Not available (requires statsmodels)")
    
    # Jarque-Bera test for normality
    jb_stat, jb_pvalue = stats.jarque_bera(residuals)
    print(f"  Jarque-Bera test p-value: {jb_pvalue:.3f}")
    print(f"  Residuals normal: {'Yes' if jb_pvalue > 0.05 else 'No'}")
    
    # Durbin-Watson test for autocorrelation
    try:
        from statsmodels.stats.stattools import durbin_watson
        dw_stat = durbin_watson(residuals)
        print(f"  Durbin-Watson statistic: {dw_stat:.3f}")
        print(f"  Autocorrelation: {'Low' if 1.5 < dw_stat < 2.5 else 'High'}")
    except ImportError:
        print("  Durbin-Watson test: Not available (requires statsmodels)")
    
    # Heteroscedasticity test (Breusch-Pagan)
    try:
        from statsmodels.stats.diagnostic import het_breuschpagan
        bp_stat, bp_pvalue, _, _ = het_breuschpagan(residuals, np.column_stack([y_pred]))
        print(f"  Breusch-Pagan test p-value: {bp_pvalue:.3f}")
        print(f"  Homoscedastic: {'Yes' if bp_pvalue > 0.05 else 'No'}")
    except (ImportError, Exception):
        print("  Breusch-Pagan test: Not available")

# Perform statistical validation for available models
for model_name, preds in models_predictions.items():
    if preds['available']:
        statistical_validation(y_test, preds['test'], model_name)

## 9. Feature Importance Analysis (for applicable models)

In [ ]:
# Feature importance analysis
print("\n🎯 FEATURE IMPORTANCE ANALYSIS")
print("=" * 35)

# For models that support feature importance
if 'lstm_model' in locals() and lstm_model is not None:
    print("\nLSTM Feature Analysis:")
    print("  LSTM uses sequential patterns in the data")
    print("  Most important: Recent sales values (last 28 days)")
    print("  Secondary: Seasonal patterns and trends")

if 'sarima_model' in locals() and sarima_model is not None:
    print("\nSARIMA Feature Analysis:")
    print("  SARIMA automatically identifies:")
    print("  - Autoregressive components (recent values)")
    print("  - Seasonal components (weekly patterns)")
    print("  - Moving average components (error correction)")
    
    # Try to get SARIMA coefficients if available
    try:
        summary = sarima_model.get_model_summary()
        print("  \nModel coefficients available in summary")
    except:
        print("  Model summary not available")

if 'prophet_model' in locals() and prophet_model is not None:
    print("\nProphet Feature Analysis:")
    print("  Prophet decomposes sales into:")
    print("  - Trend component (long-term direction)")
    print("  - Seasonal components (weekly, monthly, yearly)")
    print("  - Holiday effects (if specified)")
    print("  - Residual noise")

# Correlation analysis with features
print("\n📊 FEATURE CORRELATION ANALYSIS:")
feature_corr = product_ts_clean.corr()['sales'].abs().sort_values(ascending=False)
print("Top features correlated with sales:")
print(feature_corr.head(10))

## 10. Evaluation Summary and Recommendations

In [ ]:
# Generate comprehensive evaluation summary
print("\n📋 COMPREHENSIVE EVALUATION SUMMARY")
print("=" * 45)

# Find best performing model across different metrics
if len(detailed_df) > 0:
    test_results = detailed_df[detailed_df['Dataset'] == 'Test']
    
    best_rmse = test_results.loc[test_results['rmse'].idxmin(), 'Model']
    best_mae = test_results.loc[test_results['mae'].idxmin(), 'Model']
    best_mape = test_results.loc[test_results['mape'].idxmin(), 'Model']
    best_r2 = test_results.loc[test_results['r2_score'].idxmax(), 'Model']
    
    print(f"\n🏆 BEST MODELS BY METRIC:")
    print(f"  Best RMSE: {best_rmse}")
    print(f"  Best MAE: {best_mae}")
    print(f"  Best MAPE: {best_mape}")
    print(f"  Best R²: {best_r2}")
    
    if business_impacts:
        best_business = min(business_impacts.keys(), key=lambda x: business_impacts[x]['total_cost'])
        print(f"  Best Business Impact: {best_business}")

print("\n💡 RECOMMENDATIONS:")
print("\n1. MODEL SELECTION:")
if sarima_available:
    print("   ✅ SARIMA: Good for stationary time series with clear seasonal patterns")
    print("      - Pros: Interpretable, fast, works well with limited data")
    print("      - Cons: Assumes stationarity, limited feature integration")

if lstm_available:
    print("   ✅ LSTM: Best for complex patterns and multivariate features")
    print("      - Pros: Handles non-linear patterns, uses multiple features")
    print("      - Cons: Requires more data, less interpretable, longer training")

if prophet_available:
    print("   ✅ Prophet: Excellent for business forecasting with holidays")
    print("      - Pros: Handles holidays, robust to missing data, interpretable components")
    print("      - Cons: May overfit with limited data, less flexible")

print("\n2. IMPLEMENTATION STRATEGY:")
print("   📈 Start with SARIMA for baseline forecasts")
print("   🤖 Use LSTM when you have rich feature sets")
print("   📊 Apply Prophet for business planning and holiday impact")
print("   🔄 Consider ensemble methods combining multiple models")

print("\n3. OPERATIONAL CONSIDERATIONS:")
print("   ⏱️  Model retraining frequency: Weekly for SARIMA, Monthly for LSTM/Prophet")
print("   💾 Data requirements: Minimum 2 years for seasonal patterns")
print("   🔍 Monitor model drift using validation metrics")
print("   📊 Implement A/B testing for model selection")

print("\n4. BUSINESS IMPACT:")
if business_impacts:
    total_costs = [impact['total_cost'] for impact in business_impacts.values()]
    avg_cost = np.mean(total_costs)
    print(f"   💰 Average operational cost: ${avg_cost:,.2f}")
    print(f"   📉 Focus on reducing overstocking and understocking costs")
    print(f"   🎯 Target direction accuracy > 70% for trend following")

print("\n5. NEXT STEPS:")
print("   🔄 Scale to multiple products using the best performing model")
print("   📊 Implement automated model monitoring and alerting")
print("   🧪 Experiment with ensemble methods and advanced features")
print("   📈 Integrate with inventory management systems")

In [ ]:
# Save evaluation results
print("\n💾 SAVING EVALUATION RESULTS...")

evaluation_dir = "output/evaluation/"
os.makedirs(evaluation_dir, exist_ok=True)

# Save detailed results
if len(detailed_df) > 0:
    detailed_df.to_csv(f"{evaluation_dir}/detailed_model_evaluation.csv", index=False)
    print(f"✅ Detailed evaluation saved to {evaluation_dir}/detailed_model_evaluation.csv")

# Save business impact analysis
if business_impacts:
    business_impact_df = pd.DataFrame(business_impacts).T
    business_impact_df.to_csv(f"{evaluation_dir}/business_impact_analysis.csv")
    print(f"✅ Business impact analysis saved to {evaluation_dir}/business_impact_analysis.csv")

# Save evaluation summary
evaluation_summary = {
    'evaluation_date': datetime.now().isoformat(),
    'product_evaluated': highest_selling_product,
    'test_period': {
        'start': str(test_data.index.min()),
        'end': str(test_data.index.max()),
        'days': len(test_data)
    },
    'models_evaluated': list(models_predictions.keys()),
    'available_models': [name for name, preds in models_predictions.items() if preds['available']],
    'best_models': {
        'rmse': best_rmse if 'best_rmse' in locals() else None,
        'mae': best_mae if 'best_mae' in locals() else None,
        'mape': best_mape if 'best_mape' in locals() else None,
        'r2': best_r2 if 'best_r2' in locals() else None,
        'business_impact': best_business if 'best_business' in locals() else None
    }
}

with open(f"{evaluation_dir}/evaluation_summary.json", 'w') as f:
    json.dump(evaluation_summary, f, indent=2, default=str)

print(f"✅ Evaluation summary saved to {evaluation_dir}/evaluation_summary.json")

print("\n🎉 MODEL EVALUATION COMPLETE!")
print(f"📁 All results saved in: {evaluation_dir}")
print("📊 Review the detailed analysis above for model selection guidance")